# Used Car Prices in Saudi Arabia

## Content

1. [Import the necessary libraries](#1.-Import-the-necessary-libraries)
2. [Data Definition and Displaying](#2.-Data-Definition-and-isplaying)
3. [Data Cleaning & Preprocessing](#3.-Data-Cleaning-&-Preprocessing)
4. [Feature Engineering & Selection](#4.-Feature-Engineering-&-Selection)
5. [Machine Learning Models](#5.-Machine-Learning-Models)

## 1. Import the necessary libraries

In [ ]:
from dash import Dash, dcc, html, Input, Output, State
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## 2. Data Definition and Displaying

In [ ]:
df = pd.read_csv('used_cars.csv')


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

## 3. Data Cleaning & Preprocessing

In [ ]:
# Missing values
df.isnull().sum()

In [ ]:
median_engine_sizes = df.groupby(['Brand', 'Model'])['Engine_Size'].median()
def fill_nulls_with_median(row):
    if pd.isnull(row['Engine_Size']):
        brand = row['Brand']
        model = row['Model']
        if (brand, model) in median_engine_sizes:
            return median_engine_sizes[(brand, model)]
    return row['Engine_Size']
df['Engine_Size'] = df.apply(fill_nulls_with_median, axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
df = df[df['Price'] >= 5000]
df

In [ ]:
#Outliers
sns.boxplot(data=df['Mileage'].values);

In [ ]:
def outlier_clean(dataframe, column):
    Q1 = np.percentile(column, 25, interpolation = 'midpoint')
    Q3 = np.percentile(column, 75, interpolation = 'midpoint')
    IQR = Q3 - Q1
    lower = Q1 - 1.5*(IQR)
    higher = Q3 + 1.5*(IQR)
    print('lower bound outliers ', lower)
    print('higher bound outliers', higher)
    dataframe = dataframe.drop(dataframe[(column > higher) | (column < lower)].index, inplace=True)
outlier_clean(df, df['Mileage'])

In [ ]:
sns.boxplot(data=df['Mileage'].values);

In [ ]:
df2 = df.copy()

In [ ]:
# Price distribution 
sns.histplot(df, x='Price',kde=True);

In [ ]:
# Positive Sekewee handling 
df['Price'] = np.log(df['Price'])

In [ ]:
sns.histplot(df, x='Price',kde=True);

## 4. Feature Engineering & Selection 

In [ ]:
le_df = df

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
# categorical data

# Encoding
le_df['Brand'] = le.fit_transform(le_df['Brand'])
le_df['Model'] = le.fit_transform(le_df['Model'])
le_df['Color'] = le.fit_transform(le_df['Color'])
le_df['Options'] = le.fit_transform(le_df['Options'])
le_df['Gear_Type'] = le.fit_transform(le_df['Gear_Type'])
le_df['Region'] = le.fit_transform(le_df['Region'])

In [ ]:
#Numerical data 

# Normalisation  
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x = le_df['Mileage'].values
le_df['Mileage'] = scaler.fit_transform(x.reshape(-1,1))

In [ ]:
le_df

In [ ]:
X = le_df.drop(['Price','Year','Fuel_Type', 'Gear_Type','Region'],axis=1)
y = le_df['Price']

## 5. Machine Learning Models

### 5.1 Linear Regression 

#### Split Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40)

#### Build & Train the Model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

In [ ]:
lm.fit(X_train,y_train)

#### Test the Model

In [ ]:
print(lm.intercept_) 

In [ ]:
coeff_df = pd.DataFrame(lm.coef_,X.columns,columns=['Coefficient'])
coeff_df 

In [ ]:
predictions = lm.predict(X_test)
predictions

In [ ]:
Real_Values = np.array(y_test)
Real_Values

In [ ]:
plt.scatter(Real_Values,predictions); 
plt.plot([min(Real_Values), max(Real_Values)],
         [min(predictions), max(predictions)], color='red');

In [ ]:
sns.distplot((y_test-predictions),bins=50); # it's good if we get normal distribution

#### Evaluate the Model

In [ ]:
from sklearn import metrics
from sklearn.metrics import r2_score

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print('R^2:', r2_score(y_test, predictions))

### 5.2 Decision Tree Regression

#### Build & Train the Model

In [ ]:
from sklearn.tree import DecisionTreeRegressor  
regressor = DecisionTreeRegressor(random_state = 40) 
regressor.fit(X_train,y_train)

#### Test the Model

In [ ]:
pred = regressor.predict(X_test)
pred

In [ ]:
Real_Values = np.array(y_test)
Real_Values

In [ ]:
# from sklearn.tree import plot_tree
# plt.figure(figsize=(10,6))
# plot_tree(regressor, filled=True, feature_names=X.columns)
# plt.show()

#### Evaluate the Model

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, pred))
print('MSE:', metrics.mean_squared_error(y_test, pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred)))
print('R^2:', r2_score(y_test, pred))

In [ ]:
plt.scatter(Real_Values,pred); 

### 5.3 Random Forest Regression 

#### Train the Model 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100,random_state=0)
regressor.fit(X_train,y_train)

#### Test the Model

In [ ]:
prediction = regressor.predict(X_test)
prediction

In [ ]:
Real_Values = np.array(y_test)
Real_Values

In [ ]:
# from sklearn import tree
# first_tree = regressor.estimators_[0]
# # Plot the decision tree
# fig, ax = plt.subplots(figsize=(8, 6))
# tree.plot_tree(first_tree, ax=ax)
# plt.show()

#### Evaluate the Model

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, prediction))
print('MSE:', metrics.mean_squared_error(y_test, prediction))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, prediction)))
print('R^2:', r2_score(y_test, prediction))

In [ ]:
plt.scatter(Real_Values,prediction); 

### Compare Real Values vs Predictions on each model

In [ ]:
plt.figure(figsize=[30,30])

#first
plt.subplot(3,1,1) 
plt.scatter(Real_Values, prediction)
plt.title('Random Forest Regression')
plt.xlabel('Real Values')
plt.ylabel('Predictions')

#second
plt.subplot(3,1,2)
plt.scatter(Real_Values,predictions); 
plt.plot([min(Real_Values), max(Real_Values)],
         [min(predictions), max(predictions)], color='red');
plt.title('Linear Regression')
plt.xlabel('Real Values')
plt.ylabel('Predictions')


#third
plt.subplot(3,1,3)
plt.scatter(Real_Values,pred); 
plt.title('Decision Tree Regression')
plt.xlabel('Real Values')
plt.ylabel('Predictions')







plt.suptitle('Compare Real Values vs Predictions on each model',fontsize = 20);

## 6.Dashboard 

In [ ]:
lis = df2['Brand'].unique()

In [ ]:
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.SPACELAB, dbc.icons.BOOTSTRAP])

In [ ]:
app.layout = html.Div([
    dbc.Container(
    [
        html.H1("Used Cars in Saudi Arabia", className='custom-font mt-5 text-center'),
        html.Hr(),
    
    dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                    dbc.CardBody(
                        [
                            dbc.Row(
                                        [
                    dbc.Col(dcc.Dropdown(options=[{'label':i , 'value':i} for i in lis], 
                                         id='dropdown'),
                            width=12),
                    dbc.Col(html.H5('Car Prices Based on Some Characteristics', className="custom-font text-center mt-4"), width=6),
                    dbc.Col(html.H5('Car Models and Colors', className="custom-font text-center mt-4"), width=6),
                    dbc.Col(dcc.Graph(id='graph3'), width=6),
                    dbc.Col(dcc.Graph(id='graph2'), width=6),
                    dbc.Col(html.H5('Average Price Based on Age of The Cars', className="custom-font text-center mt-4"), width=12),
                    dbc.Col(dcc.Graph(id='graph1'), width=12),
                    
                    
                                            
                                            
            ]
        ),
                        ]),className='rounded-4 mt-3 mb-4 shadow-sm', style={}, id = 'content')
                    ),
   ], 
    ),
    ])
])
@app.callback(
    [Output('graph1', 'figure'),Output('graph2', 'figure'),Output('graph3', 'figure')],
    [Input("dropdown", "value")],
)
def render_tab_content(dropdown):
    df3 = df2.loc[(df2['Brand'] == dropdown)]
    df_fig1 = df3.groupby('Age').mean('Price')
    fig1 = px.line(df_fig1, x=df_fig1.index, y="Price", color_discrete_sequence= px.colors.qualitative.Set3,
               template= 'none')
    fig2 = px.sunburst(df3, path=['Model','Color'],color_discrete_sequence= px.colors.qualitative.Set3)
    eng_size = df3['Engine_Size'].astype(int)
    fig3 = px.scatter(df3, y='Price',x='Mileage',color='Options', size=eng_size ,
                  color_discrete_sequence= px.colors.qualitative.Set3,
                  template="plotly_white")
    
    return fig1,fig2,fig3
    
app.run_server(port=2235)